# Baseline
* Learns a user vector $u$ and an item vector $a$ and outputs $r_{ij} = u_i + a_j$
* This can be computed efficiently via Alternating Least Squares

In [ ]:
medium = ""

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");
@nbinclude("BaselineHelper.ipynb");

In [ ]:
const metric = "rating"
const name = "$medium/Baseline/$metric"
set_logging_outdir(name);

# Training

In [ ]:
# stop training once parameters have converged
@kwdef mutable struct convergence_stopper
    tolerance::AbstractFloat
    max_iters::Int
    params::AbstractVector
    prev_params::AbstractVector
    iters = 0
end

function convergence_stopper(tolerance; max_iters = Inf)
    convergence_stopper(
        tolerance = tolerance,
        max_iters = max_iters,
        params = [],
        prev_params = [],
    )
end

function stop!(x::convergence_stopper, params)
    x.iters += 1
    if x.iters > x.max_iters
        return true
    end

    if x.iters == 1
        x.params = deepcopy(params)
        return false
    end

    function maxabs(a)
        maximum(abs.(a))
    end

    x.prev_params = deepcopy(x.params)
    x.params = deepcopy(params)
    maximum(maxabs.(x.params - x.prev_params)) < x.tolerance
end;

In [ ]:
function train_model(training, stop_criteria, λ)
    GC.gc()
    λ_u, λ_a, λ_wu, λ_wa, λ_wt = λ
    users, items, ratings = training.userid, training.itemid, training.rating
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items(medium))
    weights = get_weights(λ_wu, λ_wa, λ_wt)

    ρ_u = zeros(eltype(u), length(u), num_threads())
    Ω_u = zeros(eltype(u), length(u), num_threads())
    ρ_a = zeros(eltype(a), length(a), num_threads())
    Ω_a = zeros(eltype(a), length(a), num_threads())

    p = ProgressMeter.Progress(stop_criteria.max_iters)
    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
        ProgressMeter.next!(p)
    end
    ProgressMeter.finish!(p)
    u, a
end;

# Optimize training hyperparameters

In [ ]:
function validation_mse_and_beta(λ, training, validation)
    λ = exp.(λ)
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ)
    x = make_prediction(validation.userid, validation.itemid, u, a)
    y = validation.rating
    w = get_validation_weights()
    xw = (x .* sqrt.(w))
    yw = (y .* sqrt.(w))
    β = (xw'xw + 1f-9) \ xw'yw
    loss(x * β, y, w, metric), β
end;

In [ ]:
if get_settings()["mode"] == "research"
    training = get_split(
        "training",
        metric,
        medium,
        [:userid, :itemid, :rating, :update_order, :updated_at],
    )
    validation = get_split(
        "test",
        metric,
        medium,
        [:userid, :itemid, :rating, :update_order, :updated_at],
    )
    @memoize get_training_counts(col) = get_counts(getfield(training, col))
    @memoize get_validation_weights() = powerdecay(get_counts(validation.userid), -1.0f0)
    validation_mse(λ) = validation_mse_and_beta(λ, training, validation)[1]

    res = Optim.optimize(
        validation_mse,
        fill(0.0f0, 5),
        Optim.LBFGS(),
        autodiff = :forward,
        Optim.Options(
            show_trace = true,
            extended_trace = true,
            iterations = 50,
            time_limit = 3600 * 3,
        ),
    )
    λ = exp.(Optim.minimizer(res))
    mse, β = validation_mse_and_beta(log.(λ), training, validation)
    @info "The optimal λ, β is $λ, $β found in " *
          repr(Optim.f_calls(res)) *
          " function calls"
    write_params(Dict("λ" => λ, "β" => β), name, true)
else
    @assert false
end

In [ ]:
mse, β = validation_mse_and_beta(log.(λ), training, validation)
@info "The optimal λ, β is $λ, $β found in " * repr(Optim.f_calls(res)) * " function calls"
write_params(Dict("λ" => λ, "β" => β), name, true)

## Inference

In [ ]:
function get_item_counts(medium, df)
    data = df.itemid
    counts = Dict{Int32,Int32}()
    @showprogress for i = 1:length(data)
        u = data[i]
        if u ∉ keys(counts)
            counts[u] = 0
        end
        counts[u] += 1
    end
    counts
end;

In [ ]:
training = get_split(
    "training",
    metric,
    medium,
    [:userid, :itemid, :rating, :update_order, :updated_at],
);
get_training_counts(col) = get_counts(getfield(training, col));

In [ ]:
params = read_params(name, true)
λ = params["λ"]
β = params["β"];

In [ ]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ)
item_counts = get_item_counts(medium, training);

In [ ]:
write_params(
    Dict("u" => u, "a" => a, "λ" => λ, "β" => β, "item_counts" => item_counts),
    name,
    false,
);

In [ ]:
model(users, items) = make_prediction(users, items, u, a)
write_alpha(model, medium, name, ["training", "test", "negative"]);

In [ ]:
print_losses([name], metric, medium, ["training", "test"]);